# SCIPY

Scipy, or Scientific Python, is a collection of functions to perform basic scientific programming and data analysis.

Some of its contents are:
- Fourier transforms (`scipy.fft`)
- numerical integration (`scipy.integrate`)
- interpolation tools (`scipy.interpolate`)
- linear algebra routines (`scipy.linalg`)
- signal processing tools (`scipy.signal`)
- statistical functions (`scipy.stats`)

It is also the ecosystem for several scientific packages and provides a consistent interface to the functions, avoiding duplication.  The libraries `scikit-learn` (for machine learning) and `scikit-image` (for image processing) rely on `scipy` heavily.

In [ ]:
import scipy
import matplotlib.pyplot as plt
import numpy as np

## Integrating with `scipy.integrate`

To integrate a list of numbers using the `integrate` package of `scipy`, you can use
- `trapz` (for trapezoid, the type of integration being used)
- `quad` (from quadrature) 
- etc.

In [ ]:
import scipy.integrate as integrate
result = integrate.trapz(np.array([0, 1, 2, 3, 4, 5]))
print(result)

Note that `scipy` handles NumPy arrays.

In [ ]:
data = np.arange(20).reshape((5,4))
print(data)

result = integrate.trapz(data)
print(result)

To integrate along a different axis ("dimension") of the numpy array, use the `axis` keyword (the default is `axis=1`)

In [ ]:
result = integrate.trapz(data, axis=0)
print(result)

`trapz` assumes the $y$-values we provided are taken at evenly spaced values of $x$ with $\Delta x=1$.  You can provide specific values of $x$ with the keyword `x`, as a list for example.  You can also provide a different $\Delta x$ with the keyword `dx`.

`quad` takes a function name and the limits of the integral, but no data points. It returns the integral with an estimate of the error.


In [ ]:
result = integrate.quad(np.sin, 0, np.pi)
print(result)

Consider
$$
\int_0^2 x^4\log (x+\sqrt{x^2+1}) dx\,.
$$



In [ ]:
%matplotlib inline

#import matplotlib.pyplot as plt

x = np.linspace(0, 2, 10000)

def integrand(x):
    return x**4*np.log(x+np.sqrt(x**2+1))

y = integrand(x)

plt.yscale('log')
plt.xlabel('x')
plt.ylabel('Integrand')
plt.plot(x, y, 'b-', label='Integrand')
plt.legend()

Notice that at low $x$-values the integrand hardly varies.  The `scipy.integrate.quad()` routine is **adaptive**, i.e., it adjusts the integrand evaluations to concentrate where its variations are more significant.

In [ ]:
print(integrate.quad(integrand, 0, 2))

## Integrating ODEs with `scipy.integrate.odeint`

Many physical phenomena are modeled by differential equations: oscillations of simple systems (spring-mass, pendulum, etc.), fluid mechanics (Navier-Stokes, Laplace's, etc.), quantum mechanics (Schrödinger’s), etc.  

Here is how to numerically solve equations of this kind, in relatively simple cases...

The example looks at the 1D dynamics of a spring (with rest length $L$ and elastic constant $k$) and  mass $m$ in the presence of a drag force (with drag coefficient $\beta$).  Newton’s second law reads:
$$
{{d^2x}\over {dt^2}} + {\beta \over m}{{dx}\over {dt}} + {k\over m}(x-L) = 0\,.
$$
This is a second oreder differential equation which we can rewrite as a system of two first order differential equations:
$$
\begin{align}
{{dx}\over {dt}} &= v \\
{{dv}\over {dt}} &= - {\beta \over m}{v} - {k\over m}(x-L)
\end{align}
$$
to be solved in $(x(t), v(t))$.

To do this numerically we revert to `scipy.integrate.odeint`, which has the syntax
```Python
scipy.integrate.odeint(func, y0, t, args=())
```
`func` is the **system of first order differential equations**, the array `y0` contains the initial values, time `t` is an array of time values, and arguments `args()` contains the parameters of `func` (here $L$, $k$, $m$, and $\beta$).  The unknowns in a system of differential equations are functions: `odeint` will return the values of these functions at the values provided via `t`.

In [ ]:
from scipy.integrate import odeint
 
# The system of first order ODEs: returns dx/dt, dv/dt
def odes(u, t):
    x, v = u
    return (v, (-k*(x-L) - beta*v)/m)

# Sampling times
t = np.arange(0, 20.1, 0.01)

# Initial values of x, v
u0 = np.array([1,0])
 
# Assume certain values of the physical parameters
beta = 0.4
k = 8.0
L = 0.5
m = 1.0

# Solve the ODEs
u = odeint(odes, u0, t)

# Plot x and v vs t using matplotlib
# u[:,0] is x
plt.plot(t, u[:,0], label='x')
# u[:,1] is v
plt.plot(t, u[:,1], label='v')
plt.title('Damped Oscillator')
plt.xlabel('Time')
plt.legend()
plt.show()

# Plot in phase space using matplotlib
plt.plot(u[:,0],u[:,1])
plt.title('Phase-space')
plt.xlabel('Position')
plt.ylabel('Velocity')
plt.show()

## Fitting Data with `scipy.optimize.curve_fit`

The `scipy.optimize` package provides functions for minimizing (or maximizing) objective functions, possibly subject to constraints. It includes solvers for nonlinear problems (with support for both local and global optimization algorithms), linear programing, constrained and nonlinear least-squares, root finding, and curve fitting.

A typical science use case is to fit data to a model and estimate the model parameters.  This can be easily achieved with [`scipy.optimize.curve_fit`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html).

In this example we:

1. generate some data according to a model
1. add some random noise to the data
1. write the data to file
1. read back the data with numpy
1. fit the data with `curve_fit`

### 1. Generate data

In [ ]:
nevents = 100

x = np.linspace(10, 100, nevents)

def clean_data(t, A=100, tau=10):
    return A * np.exp(-t/tau)

y = clean_data(x)

print(type(x))
print(type(y))
print(x.shape)
print(y.shape)

In [ ]:
%matplotlib inline

#import matplotlib.pyplot as plt

plt.plot(x, y, 'b-', label='Clean data')
plt.xlabel('x')
plt.legend()

### 2. Add random noise to the clean data

In [ ]:
noise = 3 * np.random.normal(size=x.size)

y_noise = y + noise

In [ ]:
#%matplotlib notebook
plt.plot(x, y_noise, 'r-', label='Clean data + noise')
plt.plot(x, y, 'g^', label='Clean data')
plt.xlabel('x')
plt.legend()

### 3. Write data to file

This time we use `numpy.savetxt` and loop over the arrays.

In [ ]:
fname = 'data.txt'

with open(fname,'w') as f:
    [f.writelines("%.5f %.5f %.5f\n" % (x[i], y[i], y_noise[i])) for i in range(len(x))]

In [ ]:
!cat data.txt

### 4. Read data from file

In [ ]:
t, z, w = np.loadtxt(fname, unpack=True)
print(t.shape)
print(z.shape)
print(w.shape)

In [ ]:
#%matplotlib notebook
plt.plot(t, z, 'r-', label='z (clean data)')
plt.plot(t, w, 'b--', label='w (clean data + noise)')

plt.xlabel('Time [s]')
plt.legend()

### 5. Fit data

`curve_fit` takes at least three arguments:
1. the function that you want to fit;
1. a set of independent data;
1. a set of dependent data.

We will keep using NumPy arrays for the data.  One can also:

4. provide initial guesses for the function parameters in a keyword `p0`;

5. give relative or absolute errors on the dependent data using the keyword `sigma` (and `absolute_sigma`).

`curve_fit` returns the list of optimal values for the parameters [the values that minimize the sum of the squared residuals model($x$) - data], and their covariance matrix.

In [ ]:
# Reminder: our clean data was generated with A * np.exp(-t/tau)
def fitfunc(x, N, alpha, c):
    return N*np.exp(-alpha*x) + c

param_labels = ['N', 'alpha', 'c']

In [ ]:
from scipy.optimize import curve_fit

fit_pars, cov_matrix = curve_fit(fitfunc, t, w)

In [ ]:
print(fit_pars)

In [ ]:
print(cov_matrix)

In [ ]:
def print_results(fit_pars, cov_matrix, param_labels):
    _ = [print(param_labels[i], ": ", fit_pars[i], "+/-", cov_matrix[i, i]**0.5) for i in range(len(fit_pars))]
    _ = [print("{0: .1f}% error on parameter {1}"
           .format(np.abs(100*cov_matrix[i, i]**0.5/fit_pars[i]), param_labels[i]))
     for i in range(len(fit_pars))]
    
print_results(fit_pars, cov_matrix, param_labels)

In [ ]:
fit_pars_with_guess, cov_matrix_with_guess = curve_fit(fitfunc, t, w, p0=[100., 0.1, 0.])

In [ ]:
print_results(fit_pars_with_guess, cov_matrix_with_guess, param_labels)

In [ ]:
#%matplotlib notebook
plt.plot(x, y, 'g--', label='Clean data')

plt.plot(t, w, 'b-', label='Data points $w$')

plt.plot(t, fitfunc(t, *fit_pars), 'm-', label='Fit N: %.2f alpha: %.3f  c: %.3f' % tuple(fit_pars))

plt.legend()
plt.xlabel('t')
plt.grid()

In [ ]:
#Clean up
!rm data.txt

## Drawing from distributions with `scipy.stats`

We will use a different package (Seaborn) to make our plot, as a reminder of the many options to do things that there are in Python.

In [ ]:
import seaborn as sns
# settings for seaborn plotting style
sns.set(color_codes=True)
# settings for seaborn plot sizes
sns.set(rc={'figure.figsize':(5,5)})

#### 1. Uniform distribution

In [ ]:
from scipy.stats import uniform

In [ ]:
nevents = 10000
start = 10
width = 20
# rvs stands for random variates
data = uniform.rvs(size=nevents, loc=start, scale=width)

In [ ]:
ax = sns.displot(data, bins=50, kde=True,
                  color='skyblue', linewidth=1, alpha=0.5)
ax.set(xlabel='Uniform Distribution ', ylabel='Frequency')

#### 2. Normal distribution

In [ ]:
from scipy.stats import norm

In [ ]:
nevents = 10000
# rvs stands for random variates
data = norm.rvs(size=nevents, loc=0, scale=1)

In [ ]:
ax = sns.displot(data, bins=50, kde=True,
                  color='skyblue', linewidth=1, alpha=0.5)
ax.set(xlabel='Normal Distribution ', ylabel='Frequency')

#### 3. Exponential distribution

In [ ]:
from scipy.stats import expon

In [ ]:
nevents = 10000
# rvs stands for random variates
data = expon.rvs(size=nevents, loc=0, scale=1)

In [ ]:
ax = sns.displot(data, bins=50, kde=True,
                  color='skyblue', linewidth=1, alpha=0.5)
ax.set(xlabel='Exponential Distribution ', ylabel='Frequency')